In [171]:
from models.stage2.transformers import BidirectionalTransformer
from models.stage1.encoder_decoder import VQVAEEncoder, VQVAEDecoder
from models.stage1.vq import VectorQuantize


In [172]:
import torch
import numpy as np
from preprocessing.data_pipeline import build_data_pipeline

from preprocessing.preprocess_ucr import UCRDatasetImporter
from pathlib import Path

from models.stage2.sample import unconditional_sample, conditional_sample

# from evaluation.evaluation import Evaluation
from utils import get_root_dir, load_yaml_param_settings, save_model
from models.stage2.maskgit import MaskGIT
import os
import tempfile
from evaluation.model_eval import Evaluation
import matplotlib.pyplot as plt


In [173]:
config_dir = "configs/config.yaml"
config = load_yaml_param_settings(config_dir)
dataset_name = config["dataset"]["dataset_name"]

dataset_importer = UCRDatasetImporter(**config["dataset"])
batch_size = config["dataset"]["batch_sizes"]["stage2"]
train_data_loader, test_data_loader = [
    build_data_pipeline(batch_size, dataset_importer, config, kind)
    for kind in ["train", "test"]
]

input_length = train_data_loader.dataset.X.shape[-1]
n_classes = len(np.unique(train_data_loader.dataset.Y.flatten()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


dim = config["encoder"]["dim"]
in_channels = config["dataset"]["in_channels"]
downsampled_width = config["encoder"]["downsampled_width"]
n_fft = config["VQVAE"]["n_fft"]

downsample_rate = 1



self.X_train.shape: (1000, 1024)
self.X_test.shape: (8236, 1024)
# unique labels (train): [0 1 2]
# unique labels (test): [0 1 2]


In [182]:
dataset_importer.X_train.shape

(1000, 1024)

In [183]:
X_train = dataset_importer.X_train[:, None, :]
X_test = dataset_importer.X_test[:, None, :]

In [184]:
X_train.shape

(1000, 1, 1024)

In [108]:
# encoder = VQVAEEncoder(
#     dim, 2 * in_channels, downsample_rate, config["encoder"]["n_resnet_blocks"]
# )
# decoder = VQVAEDecoder(
#     dim, 2 * in_channels, downsample_rate, config["decoder"]["n_resnet_blocks"]
# )
# vq_model = VectorQuantize(
#     dim, config["VQVAE"]["codebook"]["size"], **config["VQVAE"]
# )
# vq_model1 = VectorQuantize(
#     dim, config["VQVAE"]["codebook"]["size"], **config["VQVAE"]
# )

1
1


In [156]:
import numpy as np

x = torch.rand(1, 2, 4, 128)  # (batch, channels, height, width)

encoder = VQVAEEncoder(d=64, num_channels=2, downsample_rate=4, n_resnet_blocks=2)
decoder = VQVAEDecoder(d=64, num_channels=2, downsample_rate=4, n_resnet_blocks=2)
decoder.upsample_size = torch.IntTensor(np.array(x.shape[2:]))

# z = encoder(x)
# x_recons = decoder(z)

# print('x.shape:', x.shape)
# print('z.shape:', z.shape)
# print('x_recons.shape:', x_recons.shape)

In [157]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in encoder.state_dict():
    print(param_tensor, "\t", encoder.state_dict()[param_tensor].size())

print()

Model's state_dict:
num_tokens 	 torch.Size([1])
H_prime 	 torch.Size([1])
W_prime 	 torch.Size([1])
encoder.0.block.0.weight 	 torch.Size([64, 2, 3, 4])
encoder.0.block.0.bias 	 torch.Size([64])
encoder.0.block.1.weight 	 torch.Size([64])
encoder.0.block.1.bias 	 torch.Size([64])
encoder.0.block.1.running_mean 	 torch.Size([64])
encoder.0.block.1.running_var 	 torch.Size([64])
encoder.0.block.1.num_batches_tracked 	 torch.Size([])
encoder.1.block.0.weight 	 torch.Size([64, 64, 3, 4])
encoder.1.block.0.bias 	 torch.Size([64])
encoder.1.block.1.weight 	 torch.Size([64])
encoder.1.block.1.bias 	 torch.Size([64])
encoder.1.block.1.running_mean 	 torch.Size([64])
encoder.1.block.1.running_var 	 torch.Size([64])
encoder.1.block.1.num_batches_tracked 	 torch.Size([])
encoder.2.0.convs.1.weight 	 torch.Size([64, 64, 3, 3])
encoder.2.0.convs.1.bias 	 torch.Size([64])
encoder.2.0.convs.2.weight 	 torch.Size([64])
encoder.2.0.convs.2.bias 	 torch.Size([64])
encoder.2.0.convs.2.running_mean 	 tor

In [158]:
vq_model.load_state_dict(torch.load('old_models/vqmodel-FordA.ckpt'))



<All keys matched successfully>

In [94]:
for i in range(1,32):
    print(vq_model.codebook[i,:].max())
    print(vq_model.codebook[i,:].min())
    print(vq_model.codebook[i,:].norm())
    print('-------------')

tensor(3.4748)
tensor(-1.5378)
tensor(9.2859)
-------------
tensor(3.0269)
tensor(-1.2620)
tensor(8.4513)
-------------
tensor(3.9715)
tensor(-2.6926)
tensor(13.1570)
-------------
tensor(2.7962)
tensor(-1.7046)
tensor(6.9234)
-------------
tensor(5.9275)
tensor(-2.1643)
tensor(12.1612)
-------------
tensor(2.9538)
tensor(-1.2534)
tensor(7.7247)
-------------
tensor(3.5966)
tensor(-2.2370)
tensor(11.5308)
-------------
tensor(5.3751)
tensor(-3.3423)
tensor(19.6632)
-------------
tensor(5.9114)
tensor(-3.5278)
tensor(18.4416)
-------------
tensor(6.9601)
tensor(-3.5839)
tensor(20.6090)
-------------
tensor(4.1880)
tensor(-2.6006)
tensor(13.9098)
-------------
tensor(4.7466)
tensor(-2.4345)
tensor(13.1898)
-------------
tensor(2.5578)
tensor(-1.9145)
tensor(7.9467)
-------------
tensor(6.1294)
tensor(-3.4712)
tensor(20.4765)
-------------
tensor(4.3913)
tensor(-2.3702)
tensor(14.1580)
-------------
tensor(4.3634)
tensor(-1.9108)
tensor(13.0187)
-------------
tensor(4.9167)
tensor(-2.2417